In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms

import time

import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

C:\Users\nenec\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) #kernel = filter size. #out = number of filters
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
      
    
    def forward(self, t):   
    #hidden conv layers
        t = self.conv1(t) 
        t = F.relu(t) #activation function
        t = F.max_pool2d(t, kernel_size=2, stride=2)
    
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
    
    #hidden linear layers.
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
    
        t = self.fc2(t)
        t = F.relu(t)
    
    #output layer
        t = self.out(t)
  
        return t

In [4]:
USPS_transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

In [5]:
mnist = torchvision.datasets.MNIST(""
                       , train=True
                       , download=True
                       , transform = transforms.Compose([transforms.ToTensor()])
                        )

usps = torchvision.datasets.USPS("./data"
                     , train=True
                     , download=True
                     , transform = USPS_transform
                    )

In [6]:
def get_num_correct(preds, labels): 
    return preds.argmax(dim=1).eq(labels).sum().item()

In [7]:
#As a first test, let's make a prediction on an untrained network. It's expected to have e low accuracy

In [8]:
mnist_loader = torch.utils.data.DataLoader(mnist, batch_size = 100, shuffle=True)

In [9]:
network = Network()
batch = next(iter(mnist_loader))
images, labels = batch

In [10]:
images.shape #A batch is made of 100 imagines, with 1 channel color (grayscale), dim: 28x28

torch.Size([100, 1, 28, 28])

In [11]:
preds = network(images) 

In [12]:
preds.argmax(dim=1) #Values predicted by the network

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5])

In [13]:
labels #The true values of the batch

tensor([0, 7, 1, 3, 3, 5, 9, 6, 3, 5, 4, 1, 3, 1, 9, 0, 0, 3, 7, 9, 5, 5, 4, 3,
        6, 3, 6, 7, 4, 5, 0, 5, 0, 5, 4, 8, 4, 7, 1, 6, 0, 9, 8, 3, 0, 4, 6, 1,
        1, 5, 2, 3, 5, 4, 8, 2, 6, 2, 0, 1, 7, 6, 8, 2, 2, 0, 4, 8, 2, 4, 4, 9,
        4, 7, 2, 5, 6, 2, 8, 0, 0, 1, 9, 4, 9, 6, 6, 0, 4, 9, 5, 4, 1, 7, 3, 1,
        4, 3, 0, 2])

In [14]:
get_num_correct(preds, labels) #The number of correct prediction is very low (around 10% because it guesses at random!)

11

In [15]:
#NOW LET'S TRAIN OUR NETWORK ON MNIST. THE WEIGHTS AND BIASES ARE AT FIRST RANDOMIZED, BUT WITH EACH EPOCH WE LOOK FOR THE
#MINIMUM GRADIENT AND THUS MINIMIZE THE LOSS. THE OPTIMIZER TELLS US THE STEP OF THE LEARNING RATE

In [16]:
def training(network, loader, optimizer, num_epochs):
    for epoch in range(num_epochs):

        total_loss = 0
        total_correct = 0

        for batch in loader:
            images, labels = batch
    
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
    
            optimizer.zero_grad() #gradient must be reset every time, otherwise it's added to the previous one
            loss.backward()
            optimizer.step()
    
            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)
    
        print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)

In [17]:
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [18]:
mnist_training = torch.utils.data.DataLoader(mnist, batch_size = 100, shuffle=True)

In [19]:
training(network, mnist_training, optimizer, 10)

epoch: 0 total_correct: 56125 loss: 119.47598038613796
epoch: 1 total_correct: 58670 loss: 44.37592809018679
epoch: 2 total_correct: 58829 loss: 41.7818964861799
epoch: 3 total_correct: 58935 loss: 37.628778530459385
epoch: 4 total_correct: 58995 loss: 36.39221311121946
epoch: 5 total_correct: 59064 loss: 35.19196750456467
epoch: 6 total_correct: 59031 loss: 34.6643854441063
epoch: 7 total_correct: 59148 loss: 32.8998802305141
epoch: 8 total_correct: 59068 loss: 34.656969150179066
epoch: 9 total_correct: 59137 loss: 33.45586334816471


In [20]:
#Look at the loss reduction! Now the network is trained on mnist and expected to predict very well

In [21]:
mnist_test_loader = torch.utils.data.DataLoader(mnist, batch_size = 100, shuffle=True)
#Let's take another loader for the sake of shuffling

In [22]:
mnist_test = next(iter(mnist_test_loader))
images_test, labels_test = mnist_test

In [23]:
preds_test = network(images_test)

In [24]:
labels_test #The true values of the batch we are considering

tensor([5, 3, 2, 8, 9, 2, 4, 7, 0, 0, 4, 6, 2, 9, 1, 0, 5, 8, 3, 3, 7, 5, 4, 4,
        0, 1, 8, 3, 6, 4, 2, 5, 0, 5, 3, 2, 5, 9, 5, 9, 7, 5, 8, 5, 2, 5, 0, 3,
        9, 3, 4, 2, 6, 3, 8, 9, 8, 8, 6, 8, 0, 7, 2, 2, 2, 9, 1, 0, 5, 8, 3, 0,
        8, 1, 9, 3, 6, 4, 1, 1, 7, 9, 9, 7, 3, 6, 1, 1, 5, 3, 6, 3, 8, 4, 3, 0,
        1, 0, 1, 2])

In [25]:
preds_test.argmax(dim=1) #Predicted values

tensor([5, 3, 2, 8, 9, 2, 4, 7, 0, 0, 4, 6, 2, 9, 1, 0, 5, 8, 3, 3, 7, 5, 4, 4,
        0, 1, 8, 3, 6, 4, 2, 5, 0, 5, 3, 2, 5, 9, 5, 9, 7, 5, 8, 5, 2, 5, 0, 3,
        9, 3, 4, 2, 6, 3, 8, 9, 8, 8, 6, 8, 0, 7, 2, 2, 2, 9, 1, 0, 5, 8, 3, 0,
        8, 1, 9, 3, 6, 4, 1, 1, 7, 9, 9, 7, 3, 6, 1, 1, 5, 3, 6, 3, 8, 4, 3, 0,
        1, 0, 1, 2])

In [26]:
get_num_correct(preds_test, labels_test) #Wow! Now the network is predicting really well!!

100

In [27]:
### What happens now if I add USPS? ###

In [28]:
#If I make the network make a prediction on a batch from the USPS, this will predict badly because the weights and bias
#are not appropiate for it

In [29]:
usps_pretest_loader = torch.utils.data.DataLoader(usps, batch_size = 100, shuffle=True)

In [31]:
usps_pretest = next(iter(usps_pretest_loader))
usps_images_test, usps_labels_test = usps_pretest

In [32]:
usps_preds_pretest = network(usps_images_test)

In [33]:
usps_labels_test

tensor([5, 1, 6, 9, 7, 9, 4, 6, 5, 9, 8, 9, 6, 9, 9, 7, 0, 0, 6, 6, 3, 5, 1, 0,
        5, 4, 9, 6, 1, 4, 6, 9, 4, 3, 0, 0, 4, 7, 4, 3, 0, 8, 8, 1, 1, 6, 8, 2,
        4, 1, 0, 1, 2, 3, 6, 9, 7, 4, 5, 1, 7, 1, 2, 5, 1, 1, 3, 3, 7, 5, 7, 7,
        0, 5, 8, 2, 6, 2, 4, 3, 0, 2, 3, 9, 4, 0, 7, 8, 8, 6, 4, 7, 6, 0, 2, 2,
        0, 9, 1, 0])

In [34]:
usps_preds_pretest.argmax(dim=1) 

tensor([7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 8, 1, 1,
        1, 7, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])

In [36]:
get_num_correct(usps_preds_pretest, usps_labels_test)

15

In [37]:
#As expected! Our network guesses again only about 10%, because it is not trained on the usps

In [38]:
#Let's train the same network on the USPS, using the same optimizer with the same learning rate
training(network, usps_pretest_loader, optimizer, 10)

epoch: 0 total_correct: 6437 loss: 30.997983515262604
epoch: 1 total_correct: 7011 loss: 10.111000021919608
epoch: 2 total_correct: 7092 loss: 6.654686199501157
epoch: 3 total_correct: 7166 loss: 4.534430328756571
epoch: 4 total_correct: 7190 loss: 3.3648469148902223
epoch: 5 total_correct: 7209 loss: 2.930035855155438
epoch: 6 total_correct: 7198 loss: 3.0244248950621113
epoch: 7 total_correct: 7202 loss: 4.096723569207825
epoch: 8 total_correct: 7232 loss: 1.9913759651244618
epoch: 9 total_correct: 7255 loss: 1.1355954440368805


In [39]:
### question: WHY IS THE EXECUTION TIME SO QUICK WITH RESPECT TO THE TRAINING ON THE MNIST? WHY IS THE LOSS SO LOW? ###

In [40]:
#Let's see now if it's predicting better on the usps

In [41]:
usps_posttraining_loader = torch.utils.data.DataLoader(usps, batch_size = 100, shuffle=True)

In [42]:
usps_posttraining = next(iter(usps_posttraining_loader))
usps_images_post, usps_labels_post = usps_posttraining

In [43]:
usps_preds_posttraining = network(usps_images_post)

In [44]:
usps_labels_post

tensor([3, 2, 3, 8, 0, 7, 0, 0, 0, 4, 9, 8, 5, 7, 9, 0, 0, 7, 4, 9, 1, 9, 7, 8,
        0, 5, 4, 0, 2, 0, 2, 4, 7, 1, 2, 9, 7, 0, 4, 3, 2, 3, 6, 1, 4, 1, 0, 5,
        1, 2, 0, 9, 4, 1, 6, 7, 9, 3, 4, 8, 8, 0, 1, 0, 4, 0, 4, 7, 2, 4, 2, 2,
        2, 9, 1, 2, 1, 4, 8, 7, 9, 1, 9, 5, 3, 1, 2, 6, 2, 8, 9, 9, 3, 7, 0, 7,
        1, 9, 3, 0])

In [45]:
usps_preds_posttraining.argmax(dim=1)

tensor([3, 2, 3, 8, 0, 7, 0, 0, 0, 4, 9, 8, 5, 7, 9, 0, 0, 7, 4, 9, 1, 4, 7, 8,
        0, 5, 4, 0, 2, 0, 2, 4, 7, 1, 2, 9, 7, 0, 4, 3, 2, 3, 6, 1, 4, 1, 0, 5,
        1, 2, 0, 9, 4, 1, 6, 7, 9, 3, 4, 8, 8, 0, 1, 0, 4, 0, 4, 7, 2, 4, 2, 2,
        2, 9, 1, 2, 1, 4, 8, 7, 9, 1, 9, 5, 3, 1, 2, 6, 2, 8, 9, 9, 3, 7, 0, 7,
        1, 9, 3, 0])

In [46]:
get_num_correct(usps_preds_posttraining, usps_labels_post)

99

In [47]:
#YES!!! But will the network remember the MNIST now?

In [53]:
mnist_post_loader = torch.utils.data.DataLoader(mnist, batch_size = 100, shuffle=True)
mnist_test_post = next(iter(mnist_post_loader))
mnist_images_post, mnist_labels_post = mnist_test_post

In [54]:
mnist_preds_post = network(mnist_images_post)

In [55]:
mnist_labels_post

tensor([6, 0, 1, 1, 6, 7, 3, 1, 1, 1, 2, 6, 5, 8, 0, 0, 8, 2, 2, 1, 5, 6, 8, 3,
        1, 7, 9, 7, 8, 2, 3, 9, 1, 1, 8, 8, 7, 9, 4, 6, 0, 1, 9, 0, 0, 4, 9, 5,
        4, 4, 4, 1, 1, 4, 9, 7, 2, 0, 6, 6, 2, 5, 0, 8, 8, 1, 2, 5, 7, 0, 5, 4,
        7, 7, 3, 0, 0, 2, 2, 0, 6, 3, 9, 1, 5, 0, 6, 0, 7, 7, 6, 5, 2, 6, 5, 4,
        1, 1, 8, 0])

In [56]:
mnist_preds_post.argmax(dim=1)

tensor([5, 0, 1, 1, 6, 4, 3, 2, 1, 1, 2, 6, 5, 8, 7, 0, 8, 2, 2, 1, 5, 6, 8, 3,
        6, 9, 9, 8, 8, 2, 3, 9, 6, 1, 8, 8, 9, 9, 4, 6, 0, 1, 9, 0, 0, 4, 9, 5,
        4, 4, 4, 1, 1, 4, 9, 7, 2, 0, 6, 6, 2, 5, 0, 8, 8, 1, 2, 5, 9, 0, 5, 4,
        5, 9, 3, 0, 0, 2, 2, 0, 6, 3, 9, 1, 5, 0, 6, 0, 3, 5, 6, 5, 2, 6, 8, 9,
        6, 1, 8, 0])

In [58]:
get_num_correct(mnist_preds_post, mnist_labels_post)

83

In [ ]:
# THE PREDICTION ON WORSE THAN BEFORE USING USPS BUT IT'S STILL PRETTY HIGH.##
## THIS IS WHERE I WOULD START USING EWC TO HANDLE THIS CATASTROPHIC FORGETTING ##
#my guess: mnist and usps are actually pretty similar, so the network can still be able to guess both. if we were to retrain
#the network on something much different (randomly permuted mnist) it would act much worse
